Import libaries

In [1]:
import numpy as np
import pandas as pd
import pvlib
import importlib
import datetime
import pytz

Local libary

In [63]:
import av_utils as av
importlib.reload(av)

<module 'av_utils' from 'c:\\Users\\david.jung\\APV_tec_econ_calc\\APV_tec_econ_calc\\av_utils.py'>

Functions with inputs and results

In [66]:
# Location for TMY download

latitude = -15.087836  # latitude of the location (Mocambinho)
longitude = -44.015762 #
#altitude  = 454
tz= 'Brazil/East'  

# Dowload of TMY from PVGIS, altitude 
tmy, altitude = av.tmy_download(latitude, longitude, tz)

In [67]:
# Inputs for PV simulation
albedo = 0.2
track = True
pvrow_azimuth = 90
pvrow_tilt = 60
n_pvrows = 3
pvrow_width = 1
pvrow_pitch = 4
pvrow_height = 3
bifaciality = 0.9

# PV simulation (pvlib viewfactors)
pv = av.pv_yield(tmy_data = tmy, 
                albedo = albedo, 
                track = track, 
                pvrow_azimuth = pvrow_azimuth, 
                pvrow_tilt = pvrow_tilt, 
                n_pvrows = n_pvrows, 
                pvrow_width = pvrow_width, 
                pvrow_pitch = pvrow_pitch, 
                pvrow_height = pvrow_height, 
                bifaciality = bifaciality)
print("PV generation is "+str(pv.sum()/1000)+" kWh/kWp/year")

In [57]:
# Input for LCOE calculation
kWp = 100
capex = 1000         # per kWp    
opex = 20             # per kWp/y
wacc = 0.055
degre = 0.005
inflation = 0.03
N = 25

# LCOE calculation (can be externalized as functions in av_utils)
cashflow= pd.DataFrame(index=range(0,N))
cashflow["OPEX_des"] = (opex * kWp * (1+inflation)**cashflow.index) / (1+wacc)**cashflow.index
cashflow["EG_des"] = (pv.sum()/1000 * kWp * (1-degre)**cashflow.index) / (1+wacc)**cashflow.index
LCOE =  (capex * kWp + cashflow["OPEX_des"].sum() ) / cashflow["EG_des"].sum()
print("LCOE of the simulated system is "+str(round(LCOE,2))+" USD/kWh")

LCOE of the simulated system is 0.05 USD/kWh


Next steps:
- create website where inputs can be changed and results displayed (https://streamlit.io/, https://www.youtube.com/watch?v=VqgUkExPvLY) 
- Add other APV dimensions: Water costs and water protection...
